<a href="https://colab.research.google.com/github/taichihaya/test/blob/master/2_BERT_livedoor_news_add_features.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## 日本語BERTでlivedoorニュースを教師あり学習で分類

In [1]:
# 乱数シードの固定

import os
import random
import numpy as np
import torch

SEED_VALUE = 1234  # これはなんでも良い
os.environ['PYTHONHASHSEED'] = str(SEED_VALUE)
random.seed(SEED_VALUE)
np.random.seed(SEED_VALUE)
torch.manual_seed(SEED_VALUE)  # PyTorchを使う場合


### GPUの使用可能を確認

画面上部のメニュー ランタイム > ランタイムのタイプを変更 で、 ノートブックの設定 を開く

ハードウェアアクセラレータに GPU を選択し、 保存 する

In [2]:
# GPUの使用確認：True or False
torch.cuda.is_available()

# TrueならGPU使用可能

True

## 準備1：Livedoorニュースをダウンロードしてtsvファイル化

参考：https://github.com/yoheikikuta/bert-japanese/blob/master/notebook/finetune-to-livedoor-corpus.ipynb


In [3]:
# Livedoorニュースのファイルをダウンロード
! wget "https://www.rondhuit.com/download/ldcc-20140209.tar.gz"

--2020-11-30 23:27:08--  https://www.rondhuit.com/download/ldcc-20140209.tar.gz
Resolving www.rondhuit.com (www.rondhuit.com)... 59.106.19.174
Connecting to www.rondhuit.com (www.rondhuit.com)|59.106.19.174|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 8855190 (8.4M) [application/x-gzip]
Saving to: ‘ldcc-20140209.tar.gz’

ldcc-20140209.tar.g 100%[===================>]   8.44M  3.59MB/s    in 2.4s    

2020-11-30 23:27:11 (3.59 MB/s) - ‘ldcc-20140209.tar.gz’ saved [8855190/8855190]



In [4]:
# ファイルを解凍し、カテゴリー数と内容を確認
import tarfile
import os

# 解凍
tar = tarfile.open("ldcc-20140209.tar.gz", "r:gz")
tar.extractall("./data/livedoor/")
tar.close()

# フォルダのファイルとディレクトリを確認
files_folders = [name for name in os.listdir("./data/livedoor/text/")]
print(files_folders)

# カテゴリーのフォルダのみを抽出
categories = [name for name in os.listdir(
    "./data/livedoor/text/") if os.path.isdir("./data/livedoor/text/"+name)]

print("カテゴリー数:", len(categories))
print(categories)


['sports-watch', 'movie-enter', 'topic-news', 'livedoor-homme', 'smax', 'README.txt', 'it-life-hack', 'kaden-channel', 'dokujo-tsushin', 'CHANGES.txt', 'peachy']
カテゴリー数: 9
['sports-watch', 'movie-enter', 'topic-news', 'livedoor-homme', 'smax', 'it-life-hack', 'kaden-channel', 'dokujo-tsushin', 'peachy']


In [5]:
# ファイルの中身を確認してみる
file_name = "./data/livedoor/text/movie-enter/movie-enter-6255260.txt"

with open(file_name) as text_file:
    text = text_file.readlines()
    print("0：", text[0])  # URL情報
    print("1：", text[1])  # タイムスタンプ
    print("2：", text[2])  # タイトル
    print("3：", text[3])  # 本文

    # 今回は4要素目には本文は伸びていないが、4要素目以降に本文がある場合もある


0： http://news.livedoor.com/article/detail/6255260/

1： 2012-02-07T09:00:00+0900

2： 新しいヴァンパイアが誕生！　ジョニデ主演『ダーク・シャドウ』の公開日が決定

3： 　こんなヴァンパイアは見たことがない！　ジョニー・デップとティム・バートン監督がタッグを組んだ映画『ダーク・シャドウズ（原題）』の邦題が『ダーク・シャドウ』に決定。日本公開日が5月19日に決まった。さらに、ジョニー・デップ演じるヴァンパイアの写真が公開された。



In [6]:
# 本文を取得する前処理関数を定義


def extract_main_txt(file_name):
    with open(file_name) as text_file:
        # 今回はタイトル行は外したいので、3要素目以降の本文のみ使用
        text = text_file.readlines()[3:]

        # 3要素目以降にも本文が入っている場合があるので、リストにして、後で結合させる
        text = [sentence.strip() for sentence in text]  # 空白文字(スペースやタブ、改行)の削除
        text = list(filter(lambda line: line != '', text))
        text = ''.join(text)
        text = text.translate(str.maketrans(
            {'\n': '', '\t': '', '\r': '', '\u3000': ''}))  # 改行やタブ、全角スペースを消す
        return text


In [7]:
# リストに前処理した本文と、カテゴリーのラベルを追加していく
import glob

list_text = []
list_label = []

for cat in categories:
    text_files = glob.glob(os.path.join("./data/livedoor/text", cat, "*.txt"))

    # 前処理extract_main_txtを実施して本文を取得
    body = [extract_main_txt(text_file) for text_file in text_files]

    label = [cat] * len(body)  # bodyの数文だけカテゴリー名のラベルのリストを作成

    list_text.extend(body)  # appendが要素を追加するのに対して、extendはリストごと追加する
    list_label.extend(label)


In [8]:
# 0番目の文章とラベルを確認
print(list_text[0])
print(list_label[0])


東北関東大震災の影響を受け、国内ゴルフの中止が続々と決まっている。大会中に震災となり、第1ラウンド終了時で競技不成立となった「ヨコハマタイヤゴルフトーナメント PRGRレディスカップ」に加え、「Tポイントレディス」、「ヤマハレディースオープン葛城」、さらに4月8日から開催予定だった「スタジオアリス女子オープン」も、23日に中止が発表された。これで中止となったのは4大会連続となる。震災翌日には、東北高校出身の有村智恵が被災地への思いを語り、テレビカメラの前で涙を流す場面が映された。気丈に振舞う彼女を中心として、女子プロゴルファーたちは率先して各地で募金活動を行っている。有村は“ゴルフ界の浜崎あゆみ”とも呼ばれるルックスで、ゴルフファンのみならずスポーツファンの心を癒してきた。本家AYUにも負けないその笑顔とともに、一日でも早く復活してもらいたいものだ。●写真で追う！ 女性アスリート
sports-watch


In [9]:
# pandasのDataFrameにする
import pandas as pd

df = pd.DataFrame({'text': list_text, 'label': list_label})

# 大きさを確認しておく（7,376文章が存在）
print(df.shape)

df.head()


(7376, 2)


,text,label
0,東北関東大震災の影響を受け、国内ゴルフの中止が続々と決まっている。大会中に震災となり、第1ラ...,sports-watch
1,11日、国立競技場では、ロンドン五輪壮行試合＝キリンチャレンジカップ2012・U-23日本代...,sports-watch
2,20日放送、NHK「サンデースポーツ」では、山岸舞彩キャスターが日本代表・香川真司に行ったイ...,sports-watch
3,ボルフスブルクに所属する日本代表MF長谷部誠が、東日本大震災で被災した宮城県南三陸町の幼稚園...,sports-watch
4,12日深夜、フジテレビ「すぽると！」では、サッカー女子日本代表・澤穂希のインタビューが放送さ...,sports-watch


In [10]:
# カテゴリーの辞書を作成
dic_id2cat = dict(zip(list(range(len(categories))), categories))
dic_cat2id = dict(zip(categories, list(range(len(categories)))))

print(dic_id2cat)
print(dic_cat2id)

# DataFrameにカテゴリーindexの列を作成
df["label_index"] = df["label"].map(dic_cat2id)
df.head()

# label列を消去し、text, indexの順番にする
df = df.loc[:, ["text", "label_index"]]
df.head()


{0: 'sports-watch', 1: 'movie-enter', 2: 'topic-news', 3: 'livedoor-homme', 4: 'smax', 5: 'it-life-hack', 6: 'kaden-channel', 7: 'dokujo-tsushin', 8: 'peachy'}
{'sports-watch': 0, 'movie-enter': 1, 'topic-news': 2, 'livedoor-homme': 3, 'smax': 4, 'it-life-hack': 5, 'kaden-channel': 6, 'dokujo-tsushin': 7, 'peachy': 8}


,text,label_index
0,東北関東大震災の影響を受け、国内ゴルフの中止が続々と決まっている。大会中に震災となり、第1ラ...,0
1,11日、国立競技場では、ロンドン五輪壮行試合＝キリンチャレンジカップ2012・U-23日本代...,0
2,20日放送、NHK「サンデースポーツ」では、山岸舞彩キャスターが日本代表・香川真司に行ったイ...,0
3,ボルフスブルクに所属する日本代表MF長谷部誠が、東日本大震災で被災した宮城県南三陸町の幼稚園...,0
4,12日深夜、フジテレビ「すぽると！」では、サッカー女子日本代表・澤穂希のインタビューが放送さ...,0


In [11]:
# 順番をシャッフルする
df = df.sample(frac=1, random_state=123).reset_index(drop=True)
df.head()

,text,label_index
0,今回の商品だが、冒頭の写真を見てもらえばわかるので小難しい説明は不要だろう。今回紹介するのは...,5
1,こんにちは、ビデオエディターを目指している小島暢華です。さて、今回はデジタル一眼レフカメラを...,6
2,ドバイの躍動を感じる男旅的ステイもし友人にドバイを目いっぱい楽しむコツを聞かれたら、間違いな...,3
3,スマートフォン向けのゲームが増えている。この背景には従来の携帯ゲームをスマートフォン対応させ...,6
4,文字だけでは味気ないWord文書も、クリップアートを追加することで見栄えが良くなったりする。...,5


In [12]:
# 適当に説明変数作成
df["var1"] = df.text.str.contains("1")*1
df["var2"] = df.text.str.contains("2")*1
df["var3"] = df.text.str.contains("3")*1
# df = df[["text","label_index"]]

In [13]:
# tsvファイルで保存する

# 全体の2割の文章数
len_0_2 = len(df) // 5

# 前から2割をテストデータとする
test = df[:len_0_2]
test.to_csv("./test.tsv", sep='\t', index=False, header=None)
print(test.shape)

# 前2割からを訓練&検証データとする
train = df[len_0_2:]
train.to_csv("./train_eval.tsv", sep='\t', index=False, header=None)
print(train.shape)

(1475, 5)
(5901, 5)


In [14]:
# tsvファイルをダウンロードしたい場合
from google.colab import files

# ダウンロードする場合はコメントを外す
# 少し時間がかかる（4MB）
# files.download("./test.tsv")


# ダウンロードする場合はコメントを外す
# 少し時間がかかる（18MB）
# files.download("./train_eval.tsv")


## 準備2：LivedoorニュースをBERT用のDataLoaderにする

Hugginfaceのリポジトリの案内とは異なり、torchtextを使用した手法で実装

In [15]:
# MeCabとtransformersの用意
!apt install aptitude swig
!aptitude install mecab libmecab-dev mecab-ipadic-utf8 git make curl xz-utils file -y
!pip install mecab-python3==0.7
!pip install transformers==2.9.0

Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following additional packages will be installed:
  aptitude-common libcgi-fast-perl libcgi-pm-perl libclass-accessor-perl
  libcwidget3v5 libencode-locale-perl libfcgi-perl libhtml-parser-perl
  libhtml-tagset-perl libhttp-date-perl libhttp-message-perl libio-html-perl
  libio-string-perl liblwp-mediatypes-perl libparse-debianchangelog-perl
  libsigc++-2.0-0v5 libsub-name-perl libtimedate-perl liburi-perl libxapian30
  swig3.0
Suggested packages:
  aptitude-doc-en | aptitude-doc apt-xapian-index debtags tasksel
  libcwidget-dev libdata-dump-perl libhtml-template-perl libxml-simple-perl
  libwww-perl xapian-tools swig-doc swig-examples swig3.0-examples swig3.0-doc
The following NEW packages will be installed:
  aptitude aptitude-common libcgi-fast-perl libcgi-pm-perl
  libclass-accessor-perl libcwidget3v5 libencode-locale-perl libfcgi-perl
  libhtml-parser-perl libhtml-tagset-perl libhttp

In [16]:
import torch
import torchtext  # torchtextを使用
from transformers.modeling_bert import BertModel
from transformers.tokenization_bert_japanese import BertJapaneseTokenizer

# 日本語BERTの分かち書き用tokenizerです
tokenizer = BertJapaneseTokenizer.from_pretrained(
    'bert-base-japanese-whole-word-masking')


1列目：テキスト  
2列目：ラベル  
3列目以降：説明変数  
となるようなtsvを用意する

In [17]:
train_eval = pd.read_csv("./train_eval.tsv",sep="\t",header=None)
test = pd.read_csv("./test.tsv",sep="\t",header=None)
train_eval.head(2)

,0,1,2,3,4
0,ジャーナリストの田原総一朗が、「非常に面白い番組になると思います」と自信たっぷりのコメント。...,3,1,1,1
1,ゼロから始めるスマートフォンスクウェア・エニックス（スクエニ）は27日、人気のロールプレイン...,4,1,1,0


torchtextを使用して文章をID化

In [20]:
# データを読み込んだときに、読み込んだ内容に対して行う処理を定義します

max_length = 512  # 東北大学_日本語版の最大の単語数（サブワード数）は512

def tokenizer_512(input_text):
    """torchtextのtokenizerとして扱えるように、512単語のpytorchでのencodeを定義。ここで[0]を指定し忘れないように"""

    # [0]にはしない
    return tokenizer.encode(input_text, max_length=512, return_tensors='pt')

TEXT = torchtext.data.Field(sequential=True, tokenize=tokenizer_512, use_vocab=False, lower=False,
                            include_lengths=True, batch_first=True, fix_length=max_length, pad_token=0)
# 注意：tokenize=tokenizer.encodeと、.encodeをつけます。padding[PAD]のindexが0なので、0を指定します。

LABEL = torchtext.data.Field(sequential=False, use_vocab=False)

# (注釈)：各引数を再確認
# sequential: データの長さが可変か？文章は長さがいろいろなのでTrue.ラベルはFalse
# tokenize: 文章を読み込んだときに、前処理や単語分割をするための関数を定義
# use_vocab：単語をボキャブラリーに追加するかどうか
# lower：アルファベットがあったときに小文字に変換するかどうか
# include_length: 文章の単語数のデータを保持するか
# batch_first：ミニバッチの次元を用意するかどうか
# fix_length：全部の文章をfix_lengthと同じ長さになるように、paddingします
# init_token, eos_token, pad_token, unk_token：文頭、文末、padding、未知語に対して、どんな単語を与えるかを指定


In [22]:
# 各tsvファイルを読み込み、分かち書きをしてdatasetにします
# 少し時間がかかります
# train_eval：5901個、test：1475個
dataset_train_eval, dataset_test = torchtext.data.TabularDataset.splits(
    path='.', train='train_eval.tsv', test='test.tsv', format='tsv', fields=[('Text', TEXT)])

ID化されたtextと目的変数、説明変数をtensorに変換　→ TensorDatasetに格納

In [39]:
# ID化されたtextと目的変数、説明変数をtensorに変換
import torch
from torch import nn
from torch.utils.data import TensorDataset, random_split

def make_dataset_with_var(df,dataset):

  # text_idをtenosorに変換
  input_ids = []  
  for item in dataset:
    input_ids.append(padded)
    
  ids = torch.cat(input_ids,dim=0)

  # 目的変数と説明変数をtensorに変換（2列目以降）
  label_and_vars = torch.tensor(np.array(df.iloc[:,1:]))

  return TensorDataset(ids,label_and_vars)

dataset_train_eval_with_var = make_dataset_with_var(train,dataset_train_eval)
dataset_test_with_var = make_dataset_with_var(test,dataset_test)

In [49]:
# trainとvalidに分ける
train_size = int(0.75 * len(dataset_train_eval_with_var))
val_size = len(dataset_train_eval_with_var) - train_size

# データセットを分割
train_dataset, val_dataset = random_split(dataset_train_eval_with_var, [train_size, val_size])

DataLoaderにまとめる

In [50]:
from torch.utils.data import TensorDataset, random_split
from torch.utils.data import DataLoader, RandomSampler, SequentialSampler

# 訓練データローダー
train_dataloader = DataLoader(
            train_dataset,  
            sampler = RandomSampler(train_dataset), # ランダムにデータを取得してバッチ化
            batch_size = 16
        )

# 検証データローダー
validation_dataloader = DataLoader(
            val_dataset, 
            sampler = SequentialSampler(val_dataset), # 順番にデータを取得してバッチ化
            batch_size = 16
        )

In [51]:
# 辞書オブジェクトにまとめる
dataloaders_dict = {"train": train_dataloader, "val":validation_dataloader }

## 準備3：BERTのクラス分類用のモデルを用意する

Huggingfaceさんのをそのまま使うのではなく、BERTのbaseだけ使い、残りは自分で実装する

In [52]:
from transformers.modeling_bert import BertModel

# BERTの日本語学習済みパラメータのモデルです
model = BertModel.from_pretrained('bert-base-japanese-whole-word-masking')

In [53]:
num_vars = len(train.columns) - 2

In [54]:
from torch import nn

class BertForLivedoor(nn.Module):
    '''BERTモデルにLivedoorニュースの9クラスを判定する部分をつなげたモデル'''

    def __init__(self):
        super(BertForLivedoor, self).__init__()

        # BERTモジュール
        self.bert = model  # 日本語学習済みのBERTモデル

        # self.drop = nn.Dropout(p=0.3)

        # headにポジネガ予測を追加
        # 入力はBERTの出力特徴量の次元768 + 説明変数 、出力は9クラス
        self.cls = nn.Linear(in_features=768+num_vars, out_features=9)

        # 重み初期化処理
        nn.init.normal_(self.cls.weight, std=0.02)
        nn.init.normal_(self.cls.bias, 0)

    def forward(self, input_ids, vars):
        '''
        input_ids： [batch_size, sequence_length]の文章の単語IDの羅列
        '''

        # BERTの基本モデル部分の順伝搬
        # 順伝搬させる
        result = self.bert(input_ids)  # reult は、sequence_output, pooled_output

        # sequence_outputの先頭の単語ベクトルを抜き出す
        vec_0 = result[0]  # 最初の0がsequence_outputを示す
        vec_0 = vec_0[:, 0, :]  # 全バッチ。先頭0番目の単語の全768要素
        vec_0 = vec_0.view(-1, 768)  # sizeを[batch_size, hidden_size]に変換

        # [batch_size, 説明変数の数]に変換された説明変数を追加
        vec_0_and_vars = torch.cat([vec_0,vars],dim=1)
        
        output = self.cls(vec_0_and_vars)  # 全結合層

        return output


In [55]:
# モデル構築
net = BertForLivedoor()

# 訓練モードに設定
net.train()

print('ネットワーク設定完了')


ネットワーク設定完了


## 準備4：BERTのファインチューニングの設定

In [56]:
# 勾配計算を最後のBertLayerモジュールと追加した分類アダプターのみ実行

# 1. まず全部を、勾配計算Falseにしてしまう
for param in net.parameters():
    param.requires_grad = False

# 2. BertLayerモジュールの最後を勾配計算ありに変更
for param in net.bert.encoder.layer[-1].parameters():
    param.requires_grad = True

# 3. 識別器を勾配計算ありに変更
for param in net.cls.parameters():
    param.requires_grad = True


In [57]:
class LabelSmoothingCrossEntropy(nn.Module):
    def __init__(self):
        super(LabelSmoothingCrossEntropy, self).__init__()
    def forward(self, x, target, smoothing=0.1):
        confidence = 1. - smoothing
        logprobs = F.log_softmax(x, dim=-1)
        nll_loss = -logprobs.gather(dim=-1, index=target.unsqueeze(1))
        nll_loss = nll_loss.squeeze(1)
        smooth_loss = -logprobs.mean(dim=-1)
        loss = confidence * nll_loss + smoothing * smooth_loss
        return loss.mean()

In [58]:
# 最適化手法の設定
import torch.optim as optim

# BERTの元の部分はファインチューニング
optimizer = optim.Adam([
    {'params': net.bert.encoder.layer[-1].parameters(), 'lr': 5e-5},
    {'params': net.cls.parameters(), 'lr': 1e-4}
])

# 損失関数の設定
criterion = LabelSmoothingCrossEntropy() 

## 5. 訓練を実施

In [59]:
# モデルを学習させる関数を作成


def train_model(net, dataloaders_dict, criterion, optimizer, num_epochs):

    # GPUが使えるかを確認
    device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
    print("使用デバイス：", device)
    print('-----start-------')

    # ネットワークをGPUへ
    net.to(device)

    # ネットワークがある程度固定であれば、高速化させる
    torch.backends.cudnn.benchmark = True

    # ミニバッチのサイズ
    batch_size = dataloaders_dict["train"].batch_size

    # epochのループ
    for epoch in range(num_epochs):
        # epochごとの訓練と検証のループ
        for phase in ['train', 'val']:
            if phase == 'train':
                net.train()  # モデルを訓練モードに
            else:
                net.eval()   # モデルを検証モードに

            epoch_loss = 0.0  # epochの損失和
            epoch_corrects = 0  # epochの正解数
            iteration = 1

            # データローダーからミニバッチを取り出すループ
            for batch in (dataloaders_dict[phase]):
                # batchはTextとLableの辞書型変数

                # GPUが使えるならGPUにデータを送る
                inputs = batch[0].to(device)  # 文章
                labels = batch[1].to(device)  # ラベル
                vars = torch.cat(batch[2:],dim=0).view(-1,num_vars).to(device)  # 説明変数

                # optimizerを初期化
                optimizer.zero_grad()

                # 順伝搬（forward）計算
                with torch.set_grad_enabled(phase == 'train'):

                    # BERTに入力
                    outputs = net(inputs,vars)

                    loss = criterion(outputs, labels)  # 損失を計算

                    _, preds = torch.max(outputs, 1)  # ラベルを予測

                    # 訓練時はバックプロパゲーション
                    if phase == 'train':
                        loss.backward()
                        optimizer.step()

                        if (iteration % 10 == 0):  # 10iterに1度、lossを表示
                            acc = (torch.sum(preds == labels.data)
                                   ).double()/batch_size
                            print('イテレーション {} || Loss: {:.4f} || 10iter. || 本イテレーションの正解率：{}'.format(
                                iteration, loss.item(),  acc))

                    iteration += 1

                    # 損失と正解数の合計を更新
                    epoch_loss += loss.item() * batch_size
                    epoch_corrects += torch.sum(preds == labels.data)

            # epochごとのlossと正解率
            epoch_loss = epoch_loss / len(dataloaders_dict[phase].dataset)
            epoch_acc = epoch_corrects.double(
            ) / len(dataloaders_dict[phase].dataset)

            print('Epoch {}/{} | {:^5} |  Loss: {:.4f} Acc: {:.4f}'.format(epoch+1, num_epochs,
                                                                           phase, epoch_loss, epoch_acc))

    return net


In [ ]:
# 学習・検証を実行する。1epochに2分ほどかかります
import torch.nn.functional as F
num_epochs = 4
net_trained = train_model(net, dataloaders_dict,
                          criterion, optimizer, num_epochs=num_epochs)


使用デバイス： cuda:0
-----start-------
イテレーション 10 || Loss: 2.1397 || 10iter. || 本イテレーションの正解率：0.0625
イテレーション 20 || Loss: 2.0328 || 10iter. || 本イテレーションの正解率：0.3125
イテレーション 30 || Loss: 1.8565 || 10iter. || 本イテレーションの正解率：0.5
イテレーション 40 || Loss: 1.6472 || 10iter. || 本イテレーションの正解率：0.4375
イテレーション 50 || Loss: 1.4278 || 10iter. || 本イテレーションの正解率：0.75
イテレーション 60 || Loss: 1.2180 || 10iter. || 本イテレーションの正解率：0.6875
イテレーション 70 || Loss: 1.3441 || 10iter. || 本イテレーションの正解率：0.5625
イテレーション 80 || Loss: 1.3651 || 10iter. || 本イテレーションの正解率：0.5
イテレーション 90 || Loss: 1.0898 || 10iter. || 本イテレーションの正解率：0.8125
イテレーション 100 || Loss: 0.8530 || 10iter. || 本イテレーションの正解率：0.875
イテレーション 110 || Loss: 1.0088 || 10iter. || 本イテレーションの正解率：0.6875
イテレーション 120 || Loss: 0.9307 || 10iter. || 本イテレーションの正解率：0.875
イテレーション 130 || Loss: 1.0997 || 10iter. || 本イテレーションの正解率：0.6875
イテレーション 140 || Loss: 1.1255 || 10iter. || 本イテレーションの正解率：0.625
イテレーション 150 || Loss: 0.7572 || 10iter. || 本イテレーションの正解率：0.875
イテレーション 160 || Loss: 0.9648 || 10iter. || 本イテレーションの正解率：0.8

## テストデータでの性能を確認

In [ ]:
from tqdm import tqdm

# テストデータでの正解率を求める
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

net_trained.eval()   # モデルを検証モードに
net_trained.to(device)  # GPUが使えるならGPUへ送る

# epochの正解数を記録する変数
epoch_corrects = 0

for batch in tqdm(dl_test):  # testデータのDataLoader
    # batchはTextとLableの辞書オブジェクト
    # GPUが使えるならGPUにデータを送る
    device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
    inputs = batch.Text[0].to(device)  # 文章
    labels = batch.Label.to(device)  # ラベル

    # 順伝搬（forward）計算
    with torch.set_grad_enabled(False):

        # BertForLivedoorに入力
        outputs = net_trained(inputs)

        loss = criterion(outputs, labels)  # 損失を計算
        _, preds = torch.max(outputs, 1)  # ラベルを予測
        epoch_corrects += torch.sum(preds == labels.data)  # 正解数の合計を更新

# 正解率
epoch_acc = epoch_corrects.double() / len(dl_test.dataset)

print('テストデータ{}個での正解率：{:.4f}'.format(len(dl_test.dataset), epoch_acc))


100%|██████████| 93/93 [00:54<00:00,  1.72it/s]

テストデータ1475個での正解率：0.9227


https://yoheikikuta.github.io/bert-japanese/

https://github.com/yoheikikuta/bert-japanese

の「BERT with SentencePiece for Japanese text.」

では、入力テキストにタイトルを含めていますが、今回はタイトルは除いています。

同様にタイトルを抜いている、[BERTを用いた日本語文書分類タスクの学習・ハイパーパラメータチューニングの実践例](https://medium.com/karakuri/bert%E3%82%92%E7%94%A8%E3%81%84%E3%81%9F%E6%97%A5%E6%9C%AC%E8%AA%9E%E6%96%87%E6%9B%B8%E5%88%86%E9%A1%9E%E3%82%BF%E3%82%B9%E3%82%AF%E3%81%AE%E5%AD%A6%E7%BF%92-%E3%83%8F%E3%82%A4%E3%83%91%E3%83%BC%E3%83%91%E3%83%A9%E3%83%A1%E3%83%BC%E3%82%BF%E3%83%81%E3%83%A5%E3%83%BC%E3%83%8B%E3%83%B3%E3%82%B0%E3%81%AE%E5%AE%9F%E8%B7%B5%E4%BE%8B-2fa5e4299b16)でも、正解率が92%ちょっととなっており、ほぼ同じ正解率が得られました。

以上。